In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import csv
import re
import time
import random
import pandas as pd
from itertools import repeat
import os

In [2]:
#soup opener
def soup_open(url=None, decode="iso-8859-1"):
    """Open an url with BeautifulSoup and return the html

    Args:
        url (str, optional): _description_. Defaults to None.
        decode (str, optional): _description_. Defaults to "iso-8859-1".

    Returns:
        bs4.BeautifulSoup: _description_
    """
    if url is None:
        print("Add a url")

    page = urlopen(url)
    html = page.read().decode(decode)
    soup = BeautifulSoup(html, "html.parser")

    return soup

#Basic match info: 
def get_basic_match_info(soup):
    """extract the event name from a match

    Args:
        soup (bs4.BeautifulSoup): _description_

    Returns:
        str: Event name
    """
    basic_match_info = {
        "teams": None,
        "event": None,
        "tournament_instance": None,
        "type": None,
    }

    event_text = soup.find("title").get_text(strip=True)
    regex = r"^([^|]+)\|([^|]+)\|([^|]+)\|([^|]+)\|([^|]+)$"
    result = re.search(regex, event_text)

    for index, key in enumerate(basic_match_info.keys(), 1):
        basic_match_info[key] = result.group(index).strip()

    team_dict = {
        "team_a": None,
        "team_b": None,
        "team_a_tricode": None,
        "team_b_tricode": None,
        "event": None,
        "status": None,
        "bo": None,
        "date": None,
        "patch": None,
        "tournament_instance": None,
        "type": None,
    }

    event = basic_match_info["event"]

    team_dict["tournament_instance"] = basic_match_info["tournament_instance"]
    team_dict["type"] = basic_match_info["type"]

    teams_string = basic_match_info["teams"]
    pattern = r"^(.+?)\s+vs\.\s+(.+)$"

    result = re.findall(pattern, teams_string)

    teams = list(result[0]) if result else []
    team_tricodes = soup.find_all("div", {"class": "team"})

    if team_dict["team_a"] is not None:
        pass

    else:
        teamA_tricode = team_tricodes[2].get_text(strip=True)

        team_dict["team_a"] = teams[0]
        team_dict["team_a_tricode"] = teamA_tricode.strip()
        team_dict["event"] = event

    if team_dict["team_b"] is not None:
        pass
    else:
        teamB_tricode = team_tricodes[3].get_text(strip=True)

        team_dict["team_b"] = teams[1]
        team_dict["team_b_tricode"] = teamB_tricode.strip()

    match_notes = soup.find_all("div", {"class": "match-header-vs-note"})
    team_dict["status"] = match_notes[0].get_text().strip()
    team_dict["bo"] = match_notes[1].get_text().strip()[-1]

    # Header info
    header = soup.find("div", {"class": "match-header-super"})

    date = soup.find_all("div", class_="moment-tz-convert")[0].get("data-utc-ts")
    try:
        patch = header.find("div", style="font-style: italic;").get_text(strip=True)
    except Exception as e:
        print(f"Error in patch{e}")
        patch = "No patch"

    team_dict["date"] = date
    team_dict["patch"] = patch

    return team_dict


# Map draft: 
def get_map_draft(soup):
    try:
        pick_bans = soup.find(
            "div", {"class": "match-header-note"}
        ).get_text(strip=True).split(sep=";")

        pick_bans = [x.strip() for x in pick_bans]

    except Exception as e:
        print(f"Error en get_map_draft: {e}")
    
    return pick_bans

def pickBansExtractor(soup, basic_match_info=None):

    if basic_match_info is None:
        print("Add basic_match_info dict")
        
    picks_bans = get_map_draft(soup)

    dict_picks_bans = {
        "header": [
            "team",
            "rival",
            "team_1_select_1",
            "team_2_select_1",
            "team_1_select_2",
            "team_2_select_2",
            "team_1_select_3",
            "team_2_select_3",
            "decider",
            "order",
            "bo",
            "date",
            "event",
        ],
        "team_A": [],
        "team_B": [],
    }

    team_a = basic_match_info["team_a_tricode"]
    team_b = basic_match_info["team_b_tricode"]

    dict_picks_bans["team_A"].append(team_a)
    dict_picks_bans["team_A"].append(team_b)
    dict_picks_bans["team_B"].append(team_b)
    dict_picks_bans["team_B"].append(team_a)

    for element in picks_bans:
        list_element = element.split()
        if len(list_element) == 3:
            dict_picks_bans["team_A"].append(list_element[-1])
        if len(list_element) == 2:
            dict_picks_bans["team_A"].append(list_element[0])

    order_team_B = [1, 0, 3, 2, 5, 4, 6]  # for order swaping
    maps_teamA = dict_picks_bans["team_A"][2:]

    ordenado = [maps_teamA[i] for i in order_team_B]

    for map in ordenado:
        dict_picks_bans["team_B"].append(map)

    dict_picks_bans["team_A"].append("A")
    dict_picks_bans["team_B"].append("B")

    bo = basic_match_info["bo"]

    dict_picks_bans["team_A"].append(bo)
    dict_picks_bans["team_B"].append(bo)

    date = soup.find_all("div", class_="moment-tz-convert")[0].get("data-utc-ts")
    dict_picks_bans["team_A"].append(date)
    dict_picks_bans["team_B"].append(date)

    event = basic_match_info["event"]

    dict_picks_bans["team_A"].append(event)
    dict_picks_bans["team_B"].append(event)

    return dict_picks_bans

In [3]:
def check_valid_match(soup):

    event_text = soup.find("title").get_text(strip=True)
    regex = r"^([^|]+)\|([^|]+)\|([^|]+)\|([^|]+)\|([^|]+)$"
    result = re.search(regex, event_text)
    
    match_notes = soup.find_all("div", {"class": "match-header-vs-note"})
    status = match_notes[0].get_text().strip()

    if result.group(3).strip() == "Showmatch" or status != "final":
        valid_match = False
    else:
        valid_match = True
    return valid_match

def linkExtractor(url):
    """With the url from matches all in vlr, create a list of links with
    all the matches.
    """
    soup = soup_open(url)
    
    tempLink = []
    urlLinkExtract = []
    for a in soup.find_all("a", href=True):
        tempLink.append(a["href"])
        filtered_links = [link for link in tempLink if re.match(r"^/\d+", link)]

    for cleanLink in filtered_links:
        urlLinkExtract.append("https://www.vlr.gg" + cleanLink)

    return urlLinkExtract

In [156]:
#For saving 
def normalize_filename(name):
    
    name = name.lower()
    name = re.sub(r'[^\w\s-]', '', name)
    name = re.sub(r'\s+', '_', name)
    return name.strip('_')

def save_draft_to_csv(draft, url,folder="csv"):
    tournament_name = draft['team_A'][-1]  # Medio raro esto
    normalized_tournament = normalize_filename(tournament_name)

    folder_path = os.path.join(folder, normalized_tournament)
    os.makedirs(folder_path, exist_ok=True)

    file_path = os.path.join(folder_path, f'draft_{normalized_tournament}.csv')

    header = draft["header"] + ["source_url"]
    file_exists = os.path.isfile(file_path)

    with open(file_path, mode='a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(header)
        writer.writerow(draft["team_A"] + [url])
        writer.writerow(draft["team_B"] + [url])

def save_round_detail_to_csv(detail_round_dict, folder="csv", encoding='utf-8'):  # stats from the teams
    """Copy info to the dictionary"""
    tournament_name = detail_round_dict["event"][0]  # Medio raro esto
    normalized_tournament = normalize_filename(tournament_name)

    folder_path = os.path.join(folder, normalized_tournament)
    os.makedirs(folder_path, exist_ok=True)

    file_path = os.path.join(folder_path, f'round_detail_{normalized_tournament}.csv')

    header = list(detail_round_dict)
    file_exists = os.path.isfile(file_path)

    with open(file_path, "a", newline="",encoding=encoding) as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(header)

        writer.writerows(zip(*detail_round_dict.values()))
        f.close()

In [5]:
def process_match(url):
    time.sleep(random.randint(1, 2))
    soup = soup_open(url)

    if check_valid_match(soup):
        print(f"processing: {url}")
        basic_match_info = get_basic_match_info(soup)
        draft = pickBansExtractor(soup=soup, basic_match_info=basic_match_info)
        save_draft_to_csv(draft,url)
    else:
        print(f"Not valid match: {url}")

In [6]:
url = 'https://www.vlr.gg/event/matches/2282/valorant-masters-toronto-2025/?series_id=all'
lista = linkExtractor(url)
lista_recortada = lista[-4:]

In [7]:
for url in lista_recortada:
    process_match(url)

processing: https://www.vlr.gg/498633/g2-esports-vs-fnatic-valorant-masters-toronto-2025-lr3
processing: https://www.vlr.gg/498634/wolves-esports-vs-fnatic-valorant-masters-toronto-2025-lbf
Not valid match: https://www.vlr.gg/507067/team-tarik-vs-team-toast-valorant-masters-toronto-2025-showmatch
processing: https://www.vlr.gg/498628/paper-rex-vs-fnatic-valorant-masters-toronto-2025-gf


In [ ]:
basic_match_info = get_basic_match_info(soup)
draft = pickBansExtractor(soup=soup, basic_match_info=basic_match_info)

In [152]:
test = check_valid_match(soup) 

In [288]:
basic_match_info

{'team_a': 'Team Heretics',
 'team_b': 'Paper Rex',
 'team_a_tricode': 'TH',
 'team_b_tricode': 'PRX',
 'event': 'Valorant Masters Toronto 2025',
 'status': 'final',
 'bo': '3',
 'date': '2025-06-07 12:00:00',
 'patch': 'Patch 10.10',
 'tournament_instance': 'Swiss Stage',
 'type': 'Valorant match'}

In [26]:
#Test functions

#url = "https://www.vlr.gg/498633/g2-esports-vs-fnatic-valorant-masters-toronto-2025-lr3"
url ="https://www.vlr.gg/498628/paper-rex-vs-fnatic-valorant-masters-toronto-2025-gf"
#url = "https://www.vlr.gg/507067/team-tarik-vs-team-toast-valorant-masters-toronto-2025-showmatch"
soup = soup_open(url)

In [27]:
bloques = soup.find_all("div", class_="vlr-rounds-row-col")

In [ ]:
maps_played = []

map_div =soup.find_all("div", class_="map") 

for map in map_div:
    map_name_span = map.find("span", attrs={"style": "position: relative;"})
    map_name = map_name_span.find(string=True, recursive=False).strip()
    maps_played.append(map_name)

Sunset
Icebox
Pearl
Lotus


In [160]:
def round_detail_to_dict(dictRondas):
    dictFinal = {
        "teamA": [],
        "map": [],
        "side": [],
        "teamB": [],
        "rndA": [],
        "rndB": [],
        "round": [],
        "winCon": [],
        "date": [],
        "map_order": [],
        'event': []
    }

    for count, rondaAtk in enumerate(dictRondas["teamATT"]):
        dictFinal["teamA"].append(dictRondas["team_a"])
        dictFinal["teamB"].append(dictRondas["team_b"])
        dictFinal["side"].append("atk")
        dictFinal["rndA"].append(rondaAtk)
        dictFinal["rndB"].append(dictRondas["teamBCT"][count])
        dictFinal["map"].append(dictRondas["map"])
        dictFinal["round"].append(dictRondas["ratk"][count])
        dictFinal["winCon"].append(dictRondas["winConAtk"][count])
        dictFinal["date"].append(dictRondas["date"])
        dictFinal["map_order"].append(dictRondas["map_order"])
        dictFinal["event"].append(dictRondas["event"])

    for count, rondaDef in enumerate(dictRondas["teamACT"]):
        dictFinal["teamA"].append(dictRondas["team_a"])
        dictFinal["teamB"].append(dictRondas["team_b"])
        dictFinal["side"].append("def")
        dictFinal["rndA"].append(rondaDef)
        dictFinal["rndB"].append(dictRondas["teamBTT"][count])
        dictFinal["map"].append(dictRondas["map"])
        dictFinal["round"].append(dictRondas["rdef"][count])
        dictFinal["winCon"].append(dictRondas["winConDef"][count])
        dictFinal["date"].append(dictRondas["date"])
        dictFinal["map_order"].append(dictRondas["map_order"])
        dictFinal["event"].append(dictRondas["event"])

    save_round_detail_to_csv(dictFinal)

    team_b_prespective = {
        "teamA": dictFinal["teamB"],
        "map": dictFinal["map"],
        "side": ["def" if "atk" in x else "atk" for x in dictFinal["side"]],
        "teamB": dictFinal["teamA"],
        "rndA": dictFinal["rndB"],
        "rndB": dictFinal["rndA"],
        "round": dictFinal["round"],
        "winCon": dictFinal["winCon"],
        "date": dictFinal["date"],
        "map_order": dictFinal["map_order"],
        'event': dictFinal["event"]
    }

    save_round_detail_to_csv(team_b_prespective)


In [162]:
# No me gusta lo de los diccionarios y como tengo que manualmente poner las rows
def rondaDetalle(soup, basic_match_info=None, filename="rndDetail.csv", rowInfo=None):

    if basic_match_info is None:
        print("basic_match_info required")

    rondas = {
        "team_a": None,
        "team_b": None,
        "map": None,
        "teamACT": [],
        "teamATT": [],
        "teamBCT": [],
        "teamBTT": [],
        "ratk": [],
        "rdef": [],
        "winConAtk": [],
        "winConDef": [],
        "date": None,
        "map_order": None,
        "event": None,
    }

    maps = []

    map_div = soup.find_all("div", class_="map")

    for map in map_div:
        map_name_span = map.find("span", attrs={"style": "position: relative;"})
        map_name = map_name_span.find(string=True, recursive=False).strip()
        maps.append(map_name)

    bloques = soup.find_all("div", class_="vlr-rounds-row-col")
    verificador = 0
    mapNumber = 0

    rondas["date"] = basic_match_info["date"]
    rondas["map"] = maps[mapNumber]
    rondas["map_order"] = mapNumber

    rondas["event"] = basic_match_info["event"]

    for count, ronda in enumerate(bloques):
        try:
            rondas["team_a"] = basic_match_info["team_a_tricode"]
            rondas["team_b"] = basic_match_info["team_b_tricode"]
            valor = int(ronda.find_all("div", class_="rnd-num")[0].text.strip())
            imgUrl = str(ronda.find_all("img")[0])
            vict = imgUrl[0:-3].split("/")[-1].rstrip(".webp")

            if valor >= verificador:
                verificador = valor
                rondaEvaluada = re.findall(r"rnd-sq(.*)", str(bloques[count]))
                if rondaEvaluada[0] == ' mod-win mod-ct">':
                    rondas["teamACT"].append(1)
                    rondas["teamBTT"].append(0)
                    rondas["rdef"].append(valor)
                    rondas["winConDef"].append(vict)
                elif rondaEvaluada[0] == ' mod-win mod-t">':
                    rondas["teamATT"].append(1)
                    rondas["teamBCT"].append(0)
                    rondas["ratk"].append(valor)
                    rondas["winConAtk"].append(vict)
                if rondaEvaluada[1] == ' mod-win mod-ct">':
                    rondas["teamBCT"].append(1)
                    rondas["teamATT"].append(0)
                    rondas["ratk"].append(valor)
                    rondas["winConAtk"].append(vict)
                elif rondaEvaluada[1] == ' mod-win mod-t">':
                    rondas["teamBTT"].append(1)
                    rondas["teamACT"].append(0)
                    rondas["rdef"].append(valor)
                    rondas["winConDef"].append(vict)

            else:
                mapNumber += 1
                verificador = valor
                round_detail_to_dict(rondas)
                rondas = {
                    "team_a": None,
                    "team_b": None,
                    "map": None,
                    "teamACT": [],
                    "teamATT": [],
                    "teamBCT": [],
                    "teamBTT": [],
                    "ratk": [],
                    "rdef": [],
                    "winConAtk": [],
                    "winConDef": [],
                    "date": None,
                    "map_order": None,
                    "event": None,
                }

                rondas["team_a"] = basic_match_info["team_a_tricode"]
                rondas["team_b"] = basic_match_info["team_b_tricode"]

                rondas["map_order"] = mapNumber
                rondas["map"] = maps[mapNumber]
                rondaEvaluada = re.findall(r"rnd-sq(.*)", str(bloques[count]))
                imgUrl = str(ronda.find_all("img")[0])
                vict = imgUrl[0:-3].split("/")[-1].rstrip(".webp")
                fecha = basic_match_info["date"]
                rondas["date"] = basic_match_info["date"]
                rondas["event"] = basic_match_info["event"]

                if rondaEvaluada[0] == ' mod-win mod-ct">':
                    rondas["teamACT"].append(1)
                    rondas["teamBTT"].append(0)
                    rondas["rdef"].append(valor)
                    rondas["winConDef"].append(vict)
                elif rondaEvaluada[0] == ' mod-win mod-t">':
                    rondas["teamATT"].append(1)
                    rondas["teamBCT"].append(0)
                    rondas["ratk"].append(valor)
                    rondas["winConAtk"].append(vict)
                if rondaEvaluada[1] == ' mod-win mod-ct">':
                    rondas["teamBCT"].append(1)
                    rondas["teamATT"].append(0)
                    rondas["ratk"].append(valor)
                    rondas["winConAtk"].append(vict)
                elif rondaEvaluada[1] == ' mod-win mod-t">':
                    rondas["teamBTT"].append(1)
                    rondas["teamACT"].append(0)
                    rondas["rdef"].append(valor)
                    rondas["winConDef"].append(vict)
        except:
            pass

    round_detail_to_dict(rondas)
    return rondas

In [163]:
ronund = rondaDetalle(soup,basic_match_info)

In [84]:
ronund

{'team_a': 'PRX',
 'team_b': 'FNC',
 'map': 'Lotus',
 'teamACT': [1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1],
 'teamATT': [0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1],
 'teamBCT': [1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
 'teamBTT': [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
 'ratk': [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26],
 'rdef': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 25],
 'winConAtk': ['defus',
  'elim',
  'boom',
  'elim',
  'elim',
  'elim',
  'defus',
  'defus',
  'elim',
  'defus',
  'elim',
  'elim',
  'elim'],
 'winConDef': ['defus',
  'elim',
  'elim',
  'elim',
  'defus',
  'elim',
  'elim',
  'elim',
  'elim',
  'elim',
  'boom',
  'defus',
  'elim'],
 'date': '2025-06-22 13:15:00',
 'map_order': 3,
 'event': 'Valorant Masters Toronto 2025'}

In [75]:
for key in ronund.keys():
    print(key,  len(ronund[key]))

team_a 3
team_b 3
map 1
teamACT 13
teamATT 13
teamBCT 13
teamBTT 13
ratk 13
rdef 13
winConAtk 13
winConDef 13
date 1
map_order 1
event 1


In [32]:
basic_match_info = get_basic_match_info(soup)
#picks_bans = pickBansExtractor(soup, basic_match_info)

In [33]:
basic_match_info

{'team_a': 'Paper Rex',
 'team_b': 'FNATIC',
 'team_a_tricode': 'PRX',
 'team_b_tricode': 'FNC',
 'event': 'Valorant Masters Toronto 2025',
 'status': 'final',
 'bo': '5',
 'date': '2025-06-22 13:15:00',
 'patch': 'Patch 10.10',
 'tournament_instance': 'Playoffs',
 'type': 'Valorant match'}

In [ ]:
test = soup.find("div",{"class":"match-header-event-series"}).get_text(strip=True).split()

if "Showmatch" in test:
    print(True)

{'header': ['team',
  'rival',
  'team_1_select_1',
  'team_2_select_1',
  'team_1_select_2',
  'team_2_select_2',
  'team_1_select_3',
  'team_2_select_3',
  'decider',
  'order',
  'bo',
  'date',
  'event'],
 'team_A': ['G2',
  'FNC',
  'Ascent',
  'Pearl',
  'Lotus',
  'Split',
  'Icebox',
  'Sunset',
  'Haven',
  'A',
  '3',
  '2025-06-20 14:40:00',
  'Valorant Masters Toronto 2025'],
 'team_B': ['FNC',
  'G2',
  'Pearl',
  'Ascent',
  'Split',
  'Lotus',
  'Sunset',
  'Icebox',
  'Haven',
  'B',
  '3',
  '2025-06-20 14:40:00',
  'Valorant Masters Toronto 2025']}

In [5]:
dict_info = get_basic_match_info(soup)

In [6]:
dict_info

{'teams': 'G2 Esports vs. FNATIC ',
 'event': ' Valorant Masters Toronto 2025 ',
 'tournament_instance': ' Playoffs ',
 'type': ' Valorant match '}

<div class="match-header-super">
<div>
<a class="match-header-event" href="/event/2282/valorant-masters-toronto-2025/playoffs">
<img src="//owcdn.net/img/603bfd7bf3f54.png" style="height: 32px; width: 32px; margin-right: 6px;"/>
<div>
<div style="font-weight: 700;">
						Valorant Masters Toronto 2025					</div>
<div class="match-header-event-series">
						Playoffs: 
						Lower Round 3					</div>
</div>
</a>
</div>
<div style="text-align: right;">
<div class="match-header-date">
<div class="moment-tz-convert" data-moment-format="dddd, MMMM Do" data-utc-ts="2025-06-20 14:40:00">
					Friday, June 20th				</div>
<div class="moment-tz-convert" data-moment-format="h:mm A z" data-utc-ts="2025-06-20 14:40:00" style="font-size: 12px;">

							
						3:40 PM -03					</div>
<div style="margin-top: 4px;">
<div style="font-style: italic;">
								Patch 10.10							</div>
</div>
</div>
</div>
</div>

In [42]:
header = soup.find("div", {"class": "match-header-super"})

# Extraer texto limpio de los elementos deseados
event_name = header.find("div", style="font-weight: 700;").get_text(strip=True)
event_stage = header.find("div", class_="match-header-event-series").get_text(strip=True)

date = header.find("div", {"data-moment-format": "dddd, MMMM Do"}).get_text(strip=True)
time = header.find("div", {"data-moment-format": "h:mm A z"}).get_text(strip=True)

patch = header.find("div", style="font-style: italic;").get_text(strip=True)

In [ ]:
date+" "+time

'3:40 PM -03'